In [17]:
import os
import pandas as pd


In [18]:
# list of the names of the feature sin the dataset
feature_names = ['stream', 'slope', 'elevation', 'vegetation']

# dictionary object that lists for each feature (key) the set of levels in the domain of the feature (value)
feature_levels = {'stream':['false','true'], 
                 'slope':['flat','moderate','steep'], 
                 'elevation':['low','medium','high','highest'], 
                 'vegetation':['chaparral','riparian','conifer']}



# Then we create our dataset of training instances
# the first row in the dataset (dataset[0]) lists the names of the features

dataset=[['false','steep','high','chaparral'],
          ['true','moderate','low','riparian'],
          ['true','steep','medium','riparian'],
          ['false','steep','medium','chaparral'],
          ['false','flat','high','conifer'],
          ['true','steep','highest','conifer'],
          ['true','steep','high','chaparral']]

In [19]:
# 1. The name of the target feature
# The target feature name is listed in the last element of feature_names
print("Target feature: " + str(feature_names[-1:]))

# 2. List the target feature values for the instances in the dataset
for i in range(0, len(dataset)):
    print("Instance " + str(i+1) + " Target Feature Value = " + str(dataset[i][-1:]))

# 3. The description of the 3rd instance in the dataset
# Remember, Python lists are zero indexed.  
print("Instance 3 = " + str(dataset[2]))


# 4. The value of the 'elevation' feature for the 5th instance in the dataset
index = feature_names.index('elevation')
print("The elevation feature for instance 5 in the datset is = " + str(dataset[4][index]))

Target feature: ['vegetation']
Instance 1 Target Feature Value = ['chaparral']
Instance 2 Target Feature Value = ['riparian']
Instance 3 Target Feature Value = ['riparian']
Instance 4 Target Feature Value = ['chaparral']
Instance 5 Target Feature Value = ['conifer']
Instance 6 Target Feature Value = ['conifer']
Instance 7 Target Feature Value = ['chaparral']
Instance 3 = ['true', 'steep', 'medium', 'riparian']
The elevation feature for instance 5 in the datset is = high


In [20]:
#This function takes a list of values as input and return the entropy of the list
    
def entropy(values):
    
   from math import log
   val_counts={}
   for v in values:
        if v in val_counts.keys():
            count = val_counts[v]
            val_counts[v]=count+1
        else:
            val_counts[v]=1
   entropy=0.0
   for v in val_counts.keys():
      p=float(val_counts[v])/len(values)
      entropy=entropy+(p*log(p,2))
   if entropy!=0:
        entropy=entropy*-1
   return (entropy)

In [21]:
animalset1 = ['cat', 'horse', 'horse', 'horse', 'horse', 'horse', 'horse', 'horse', 'horse', 'horse', 'horse']
print("Entropy animalset1 = " + str(entropy(animalset1)))

animalset2 = ['horse', 'horse', 'horse', 'horse', 'horse', 'dog', 'dog', 'dog', 'dog', 'dog']
print("Entropy animalset2 = " + str(entropy(animalset2)))

animalset3 = ['horse', 'horse', 'dog', 'dog', 'cat', 'cat', 'sheep', 'sheep', 'lion', 'lion']
print("Entropy animalset3 = " + str(entropy(animalset3)))

Entropy animalset1 = 0.4394969869215134
Entropy animalset2 = 1.0
Entropy animalset3 = 2.321928094887362


In [22]:
#The following function extracts a column of values from a table and returns it as a list

def get_feature_column(featureindex=-1, dataset=None):
    featureColumn = []
    for i in range(0,len(dataset)):
        featureColumn.append(dataset[i][featureindex])
    return(featureColumn)


In [23]:
# Let's test it.
# We assume that the target feature is the rightmost column in the dataset
# so to get the index of this feature we simply subtract 1 from the length of the first row in the dataset. 

target_index = len(dataset[0])-1


# We can then use the get_feature_column function to extract the feature column into a list

target_column = get_feature_column(target_index,dataset)
print("Target column = " + str(target_column))
print("Entropy of dataset1: " + str(entropy(target_column)))

Target column = ['chaparral', 'riparian', 'riparian', 'chaparral', 'conifer', 'conifer', 'chaparral']
Entropy of dataset1: 1.5566567074628228


In [24]:
# Write the function 
def create_partitions(featureIndex=-1,dataset=None):
    partitions={}
    for i in range(0, len(dataset)):
        tmpValue = dataset[i][featureIndex]
        tmpList = []
        if tmpValue in partitions.keys():
            tmpList = partitions[tmpValue]
            tmpList.append(dataset[i])
        else:
            tmpList.append(dataset[i])
        partitions[tmpValue]=tmpList
    return(partitions)
    
    
def calculate_remainder(partitions):
    remainder=0
    #we assume that the target feature is the rightmost column in an instance
    #so we can get the target index by retrieveing an one of the value from the
    #partitions datasture (next(iter(partitions.values()))) and subtracting 1
    #from the length of this instance.
    exampleInstance=(next(iter(partitions.values())))[0]
    targetIndex=len(exampleInstance)-1
    #in order to be able to weight the entropy of each partition
    #we need to know the total number of examples across all the partitions
    #this number defines the denominator in the weight term
    #we store this number in the variable size_dataset
    size_dataset =0
    for k in partitions.keys():
        size_dataset = size_dataset + len(partitions[k])
    #we are no ready to calculate the remaining entropy by calculating a
    #the weighted sum of the entropy for each partition
    for k in partitions.keys():
        #calculate the entropy for each partition
        tmpPartition = partitions[k]
        targetColumn = get_feature_column(targetIndex,tmpPartition)
        ent = entropy(targetColumn)
        #calculate the weight for each partition
        weight = len(tmpPartition)/size_dataset
        #sum the weighting remaining entropy for each partition
        remainder = remainder + (weight * ent)
    return(remainder)


In [25]:
#use the create_paritions function to split the dataset using descriptive feature 'slope' (feature index = 1)
slope_partitions = create_partitions(1,dataset)
print("Paritions created:")
print("---------------------")

for k in slope_partitions.keys():
    print(str(k)+ ":")
    instances = slope_partitions[k]
    for inst in instances:
        print('\t\t' +str(inst))

print("---------------------")

    
#use the calculate_remainder function to calculate the entropy remaining after we split dataset using the slop feature
rem = calculate_remainder(slope_partitions)
print("Remaining entropy after partitioning: " + str(rem))

Paritions created:
---------------------
steep:
		['false', 'steep', 'high', 'chaparral']
		['true', 'steep', 'medium', 'riparian']
		['false', 'steep', 'medium', 'chaparral']
		['true', 'steep', 'highest', 'conifer']
		['true', 'steep', 'high', 'chaparral']
moderate:
		['true', 'moderate', 'low', 'riparian']
flat:
		['false', 'flat', 'high', 'conifer']
---------------------
Remaining entropy after partitioning: 0.9792504246104776


In [26]:
def information_gain(feature_index=-1, dataset=[]):

    #calculate the entropy of the dataset before we partition it using the feature 

    target_index = len(dataset[0])-1
    target_column = get_feature_column(target_index,dataset)
    
    h = entropy(target_column)

    #calculate the remaining entropy after we partition the dataset using the feature
    partitions = create_partitions(feature_index,dataset)

    rem = calculate_remainder(partitions)

    #calculate the information gain for the feature
    ig = h - rem
 
    return(ig)

In [27]:
# Iterate across the indexes of the descriptive features in the dataset
# (we assume that the target feature is the last feature in the dataset)
# and call the information_gain function for each descriptive feature by passing in the feature index

for i in range(0, len(dataset[0])-1):

    ig = information_gain(i,dataset)

    print("The information gain for feature " + feature_names[i] + " is: " + str(ig))

The information gain for feature stream is: 0.30595849286804166
The information gain for feature slope is: 0.5774062828523452
The information gain for feature elevation is: 0.8773870642966131


In [28]:
class tree_node:
  def __init__(self, feature_name='', feature_index='', branches={}, instances=[], prediction=''):

    self.feature_name=feature_name    #stores the name of the feature tested at this node
    self.feature_index=feature_index  #stores the index of the feature column in the dataset
    self.branches=branches          #a dictionary object: each key=level of test feature, each value=child node of this node
    self.instances=instances        #in a leaf node this list stores the set of instances that ended up at the leaf
    self.prediction=prediction      #in a leaf node this variable stores the target level returned as a prediction by the node


In [29]:
# return true if all the instances in the dataset D have the same target level
# A handy way to check for this condition is by checking if the entropy of the 
# aataset with respect to the target feature == 0

def all_same(D=[]):

    if len(D) > 0:
        target_index = len(D[0])-1
        target_column = get_feature_column(target_index,D)

        if entropy(target_column) == 0:
            return True

    return False

    

#return the majority target level in the instances list
def majority_target_level(D):

        #assume the target feature is the last feature in each instance
        target_index = len(D[0])-1

        #extract the set of target levels in the instances at this node
        target_column = get_feature_column(target_index,D)

        #create a dictionary object that records the count for each target level
        levels_count = {}

        for l in target_column:
            if l in levels_count.keys():
                levels_count[l]+=1
            else:
                levels_count[l]=1

        #find the target level with the max count
        #for ease of implementation we break ties in max counts
        #by symply returning the first level we find with the max count

        max_count = -999999
        majority_level = ''

        for k in levels_count.keys():
            if levels_count[k] > max_count:
                max_count=levels_count[k]
                majority_level=k

        return(majority_level)

In [30]:
# This **ID3** implementation takes 5 parameters. 
# The 5 parameters are as follows:
#1. d = list of descriptive features not yet used on the path from the root to the current node 
#2. D = the set of training instances that have descended the path to this node
#3. parentD = the set of training instances at the parent of this node
#4. feature_levels = a dictionary object that lists for each feature (key) the set of levels in the domain of the feature (value)
#5. feature_names = a list of the names of the features in the dataset

def id3(d=[], D=[], parentD=[], feature_levels={}, feature_names=[]):

    if all_same(D):

        return tree_node(feature_name='',feature_index=-1,branches={},instances=D,prediction=D[0][len(D[0])-1])

    elif len(d) == 0:

        return tree_node(feature_name='',feature_index=-1,branches={},instances=D,prediction=majority_target_level(D))

    elif len(D) == 0:

        return tree_node(feature_name='',feature_index=-1,branches={},instances=D,prediction=majority_target_level(parentD))

    else:

        d_best = ""
        best_index = -1
        max_IG = -9999

        for f in d:

            feature_index = feature_names.index(f)
            tmp_IG = information_gain(feature_index,D)

            if tmp_IG > max_IG:
                max_IG = tmp_IG
                d_best = f
                best_index=feature_index

        node = tree_node(feature_name=d_best,feature_index=best_index,branches={},instances=[],prediction='')

        #partition the dataset using the feature with the highest information gain
        partitions=create_partitions(best_index,D)

        #remove d_best from the list of features passed down to the children of this node
        d_new = [ f for f in d if not f.startswith(d_best) ]

        #iterate across all the levels of the feature and create a branch for each level
        #we use arg4 for this because it may be that one or more of the levels of the feature do not appears in D

        for level in feature_levels[d_best]:

            if level in partitions.keys():
                d_new_1 = partitions[level]

            else:

                #if there is a feature level that does not occur in D
                #then create a child node where the set of training instances
                #at the node is empty

                d_new_1 = []

            node.branches[level]=id3(d_new,d_new_1,D,feature_levels,feature_names)

        return(node)

In [31]:
#featureNames[:-1] is the list of features names in the dataset apart from the last feature (i.e., excluding the target feature)

tree = id3(feature_names[:-1], dataset, dataset, feature_levels, feature_names)

In [32]:
#This function prints out the tree in text format

def print_tree(tree, indent='-'):

    if tree.prediction == '':

        indent+="--"
        for level in tree.branches.keys():
            print(indent+tree.feature_name + ':' + str(level))
            print_tree(tree.branches[level],indent)
    else:

        s = ''
        for c in indent:
            s+=' '
        print(s+" prediction = " + tree.prediction)

        
#Here we call the function and pass in the tree we want to output
print_tree(tree,"")

--elevation:low
   prediction = riparian
--elevation:medium
----stream:false
     prediction = chaparral
----stream:true
     prediction = riparian
--elevation:high
----slope:flat
     prediction = conifer
----slope:moderate
     prediction = chaparral
----slope:steep
     prediction = chaparral
--elevation:highest
   prediction = conifer


In [33]:
#This function returns a prediction from a tree for a query instance
def make_prediction(query, tree, feature_levels):

    if tree.prediction != '':

        #if we have reached a leaf node return the prediction
        return tree.prediction

    else:

        #otherwise descend the tree.
        #1. get the level of the query instance for the node test feature

        level = query[tree.feature_index]
        for l in feature_levels[tree.feature_name]:

            if l.startswith(level):
                #2. find the branch that matchs this level and desencd the branch
                return make_prediction(query,tree.branches[level],feature_levels)

        print("No prediction!")


In [34]:
query1 = ['true','moderate','low','?']
print("Query: " + str(query1) + " Prediction: " + make_prediction(query1, tree, feature_levels))

query2 = ['true','moderate','medium','?']
print("Query: " + str(query2) + " Prediction: " + make_prediction(query2, tree, feature_levels))

query3 = ['true','moderate','highest','?']
print("Query: " + str(query3) + " Prediction: " + make_prediction(query3, tree, feature_levels))

query4 = ['true','moderate','high','?']
print("Query: " + str(query4) + " Prediction: " + make_prediction(query4, tree, feature_levels))

query5 = ['true','steep','high','?']
print("Query: " + str(query5) + " Prediction: " + make_prediction(query5, tree, feature_levels))

query6 = ['true','flat','high','?']
print("Query: " + str(query6) + " Prediction: " + make_prediction(query6, tree, feature_levels))

Query: ['true', 'moderate', 'low', '?'] Prediction: riparian
Query: ['true', 'moderate', 'medium', '?'] Prediction: riparian
Query: ['true', 'moderate', 'highest', '?'] Prediction: conifer
Query: ['true', 'moderate', 'high', '?'] Prediction: chaparral
Query: ['true', 'steep', 'high', '?'] Prediction: chaparral
Query: ['true', 'flat', 'high', '?'] Prediction: conifer
